Autores:
1. David Montaño Castro
2. Sergio Israel Durán García.

<h1><center>CASO DE ESTUDIO : ANÁLISIS COMPARATIVO DE LA ESPERANZA DE VIDA Y SOBRE MORTALIDAD MASCULINA POR MEDIO DE POLINOMIOS DE NEWTON</center></h1> 

### Funciones auxiliares

#### Polinomio de Newton

In [1]:
M_Dif=function(xi,fxi){
  r1=length(xi)
  r2=length(fxi)
  
  if(r1!=r2){
    print("el metodo no se puede aplicar")
    break
  } else {mat=matrix(0,r1,r2)
  for (i in 1:r1){
    mat[i,1]=fxi[i]
  }
  for(i in 2:r1){
    for (j  in 2:i){
      mat[i,j]=(mat[i,(j-1)]-mat[(i-1),(j-1)])/(xi[i]-xi[i-(j-1)])
      
    }
  }
  return(mat)}
}
I_N=function(A,xi,r){
  a=dim(A)
  a1=a[1]
  a2=a[2]
  
  v=rep(0,a1)
  for(i in 2:a1){
    prod=1
    for(j in 1:(i-1)){
      prod=(prod*(r-xi[j]))
    }
    v[(i-1)]=prod
  }
  
  inter=A[1,1]
  for(i in 2:a1){
    inter=inter+(A[i,i])*v[(i-1)]
    
  }
  return(inter)
}


### Ajustes previos


In [2]:
setwd("C:/Users/actda/OneDrive/Documentos/SEMESTRE 2021-2/Análisis Numérico/ProyectoFinal")
options(scipen = 999)
library("ggplot2")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


### Importar datos
Toda la información fue consultada en las bases de datos del [INEGI](https://www.inegi.org.mx/)

In [3]:
life_ex = read.csv("lif_ex.csv")
sobremorta = read.csv("Sobremortalidad.csv")

INTRODUCCIÓN
---
 Para la demografía, los métodos numéricos representan una herramienta fundamental, derivado de la dificultad de recopilar con exactitud las estadísticas vitales y todos los parámetros, como son :  años persona vividos y población a edad exacta (x). Estas son estimadas, por ejemplo, usando técnicas vistas en este curso. En este proyecto utilizamos el interpolador de Newton para estimar dos parámetros que reflejan en esencia la esperanza de vida y la sobremortalidad masculina, de acuerdo a nuestros resultados obtenidos, compararemos los valores que el sitio oficial del INEGI registró hasta el año 2019.


### Objetivo del análisis
Elaborar una estimación intervalar por medio de polinomios de Newton para los parámetros de esperanza de vida en hombres (2010-2019) y sobremortalidad masculina (1994-2019) para los años impares.

### ¿Qué es la esperanza de vida?
Según el [INEGI](http://cuentame.inegi.org.mx/poblacion/esperanza.aspx?tema=P;) : Se refiere al número de años que en promedio se espera que viva una persona después de nacer. Una esperanza de vida alta indica un mejor desarrollo económico y social en la población.

### ¿Qué es la sobre mortalidad?
Según el [INEGI](https://www.inegi.org.mx/temas/mortalidad/) : 	Defunciones masculinas por cada 100 femeninas.

### Estructura de la base de datos _life_ex_
__life_ex__ contiene una serie de tiempo anual. Esta está conformada por 2 columnas:
1. __anos (años)__ : Registro de años, abarcando del 2010 al 2020.
2. __esperanza (Esperanza de vida)__ : La esperanza de vida correspondiente a cada año.

__NOTA__ : Las esperanzas de vida son para hombres.

In [4]:
life_ex

ï..aÃ.os,esperanza_vida
2010,71.6
2011,71.8
2012,72.0
2013,72.1
2014,72.1
2015,71.9
2016,72.0
2017,72.0
2018,72.2
2019,72.2


Para su análisis, se ha decidido dividirla:
1. __life_ex_par__ : Contiene los años pares entre los periodos 2010 y 2020.

In [7]:
life_ex_par

,anos,esperanza
1,2010,71.6
3,2012,72.0
5,2014,72.1
7,2016,72.0
9,2018,72.2
11,2020,72.4


2. __life_ex_impar__: Contiene los años impares entre los periodos 2010 y 2020.

In [8]:
life_ex_impar[,c(1,2)]

,anos,esperanza
2,2011,71.8
4,2013,72.1
6,2015,71.9
8,2017,72.0
10,2019,72.2


El polinómio se construirá ocupando __life_ex_par__ en aras de poder estimar de manera exitosa los valores de __life_ex_impar__ en vista de que las estimaciones tienen que estar dentro del rango de datos con el que se constituyó el _spline_.

### Estructura de la base de datos _sobremorta_
__sobremorta__ contiene una serie de tiempo anual. Esta está conformada por 2 columnas:
1. Ano (años) : Registro de años, abarcando de 1994 al 2019.
2. Sobremortalidad (Sobre mortalidad) : La sobre mortalidad correspondiente a cada año.

__NOTA__ : La sobre mortalidad siempre es un cálculo específico para hombres.

In [ ]:
sobremorta

Para su análisis, se ha decidido dividirla:
1. __sobremorta_par__ : Contiene los años pares entre los periodos 1994 y 2018

In [ ]:
sobremorta_par

2. __sobremorta_impar__: Contiene los años impares entre los periodos 1994 y 2018 

In [ ]:
sobremorta_impar[,c(1,2)]

El polinómio se construirá ocupando __sobremorta_par__ en aras de poder estimar de manera exitosa los valores de __sobremorta_impar__ en vista de que las estimaciones tienen que estar dentro del rango de datos con el que se constituyó el _spline_.

__ESPERANZA DE VIDA PARA HOMBRES__
---

In [6]:
# Cambiar el nombre de las columnas.
colnames(life_ex) = c("anos","esperanza")

# Seleccionar años pares.
life_ex_par = life_ex[seq(1,11,2),c(1,2)]
life_ex_impar = life_ex[seq(2,10,2),c(1,2)]

#Implementar modelo de Newton en valores pares.
MD_life_ex = M_Dif(life_ex_par$anos,life_ex_par$esperanza)

#Calcular cada valor estimado y guardarlo en life_ex_impar como columna
estimaciones_life_ex = c()
for(i in life_ex_impar$anos){
  estimaciones_life_ex = c(estimaciones_life_ex,I_N(MD_life_ex,life_ex_par$anos,i))
}

#Agregar columna de las estimaciones.
life_ex_impar$Estimaciones = estimaciones_life_ex

#Agregar columna de error relativo.
life_ex_impar$Error_rel = abs(life_ex_impar$esperanza- life_ex_impar$Estimaciones)


### ¿Qué tan bien puede estimar el polinómio de Newton la esperanza de vida?
Los valores estimados son muy simialres a los originales. Las dos estimaciones que menos aproximadas se encuentran corresponden a los años __2015 y 2019__, pues difieren en __0.1390625 y 0.1828125__ unidades respectivamente.   

In [ ]:
life_ex_impar

p = ggplot() + geom_line(data = life_ex_impar, aes(x = anos, y = esperanza), size = 1, colour = "green") + geom_point(data = life_ex_impar, aes(x = anos, y = esperanza),size = 3, colour = "orange") +
geom_line(data = life_ex_impar, aes(x = anos, y = Estimaciones), size = 1, colour = "red") + geom_point(data = life_ex_impar, aes(x = anos, y = Estimaciones),size = 3, colour = "orange") 

#Imprimer la grafica
p + xlab("Periodos") + ylab("Esperanza de vida (años)") + ggtitle("Serie de tiempo sobre Esperanza de vida masculina (años)") + theme(plot.title = element_text(hjust = 0.5))

__SOBRE MORTALIDAD PARA HOMBRES__
---

In [ ]:
# Seleccionar años pares.
sobremorta_par = sobremorta[seq(1,26,2),c(1,2)]
sobremorta_impar = sobremorta[seq(2,24,2),c(1,2)]

#Implementar modelo de Newton en valores pares.
MD_sobremorta = M_Dif(sobremorta_par$Ano,sobremorta_par$Sobremortalidad)

#Calcular cada valor estimado y guardarlo en sobremorta_impar como columna
estimaciones_sobremorta = c()
for(i in sobremorta_impar$Ano){
  estimaciones_sobremorta = c(estimaciones_sobremorta,I_N(MD_sobremorta,sobremorta_par$Ano,i))
}

#Agregar columna de las estimaciones.
sobremorta_impar$Estimaciones = estimaciones_sobremorta

#Agregar columna de error relativo.
sobremorta_impar$Error_rel = abs(sobremorta_impar$Sobremortalidad- sobremorta_impar$Estimaciones)

### ¿Qué tan bien puede estimar el polinómio de Newton la sobre mortalidad?
Esta estimación presenta mayor variabilidad, teniendo su mayor diferencia en los años __1995 y 2017__ con __5.08 y 3.34__ unidades.

In [ ]:
sobremorta_impar

p = ggplot() + geom_line(data = sobremorta_impar, aes(x = Ano, y = Sobremortalidad), size = 1, colour = "green") + geom_point(data = sobremorta_impar, aes(x = Ano, y = Sobremortalidad),size = 3, colour = "orange") +
geom_line(data = sobremorta_impar, aes(x = Ano, y = Estimaciones), size = 1, colour = "red") + geom_point(data = sobremorta_impar, aes(x = Ano, y = Estimaciones),size = 3, colour = "orange") 

#Imprimer la grafica
p + xlab("Periodos") + ylab("Sobre mortalidad") + ggtitle("Serie de tiempo sobre Sobre Mortalidad") + theme(plot.title = element_text(hjust = 0.5))

Conclusión
---

De los datos obtenidos podemos concluir que para el estimador de mortalidad, el sesgo presentado es menor, es decir, nos acercamos con mayor eficacia  a los datos que el INEGI arroja; por otro lado, nuestra estimación en torno a la sobre mortalidad presenta variaciones significativas, pues al provenir de menos nodos y dado su comportamiento caótico que presentan las muertes masculinas fue más difícil para nosotros acercarnos,  en la medida de lo posible, a los valores “reales”.